In [2]:
import sys
sys.path.append("../")
import consts
import api
from subset import addr_operation
from Anonymizer import anonymizer
import pickle
from copy import deepcopy

attr_list = consts.ATTR_LIST
attr_list.append('seq')

original_file = '../evaluation/original_data.csv'
anonymized_file = '../' + consts.ORIGIN_FILE
watermarked_file = '../' + consts.MODIFIED_FILE

csv_header, org_list = api.parsed_list(original_file, True)
_, anonymized_list = api.parsed_list(anonymized_file, True)
_, watermarked_list = api.parsed_list(watermarked_file, True)

In [37]:
def calc_addr_tree(addr_tree, addr, addr_attr):
    for addr_freq in addr_tree:
        if addr == addr_freq[0]:
            addr_freq[1] += 1
            break
    else:
        addr_tree.append([addr, 1])
    if addr == ['関東', '*', '*', '*', '*']:
        return
    else:
        calc_addr_tree(addr_tree, anonymizer.address_masking(addr_attr, deepcopy(addr)), addr_attr)

In [31]:
def addr_tree(addr_l):
    # treeじゃないけど
    # addr_tree = [addr_freq, ...]
    # addr_freq = [[listed_generalized_addr], freq]
    addr_attr = ['addr0', 'addr1', 'addr2', 'addr3', 'addr4']
    
    addr_tree = list()
    
    for addr in addr_l:
        calc_addr_tree(addr_tree, addr, addr_attr)
    return addr_tree

In [38]:
def loss(attr_type, tree, bottom_attr, top_attr=None):
    if top_attr is not None:
        return
    return

In [60]:
import unicodedata
def left(digit, msg):
    for c in msg:
        if unicodedata.east_asian_width(c) in ('F', 'W', 'A'):
            digit -= 2
        else:
            digit -= 1
    return msg + ' '*digit

# refer to http://d.hatena.ne.jp/naoya/20090412/btree
class Node:
    def __init__(self):
        self.value = None
        self.leaf_num = 0
        self.children = []
        self.is_leaf = False

    def insert(self, v):
        s = Node()
        s.value = v
        self.children.append(s)

    def show(self, pad):
        print(
            "%s, %s, %d" %
            (left(35, '='*pad + self.value),
             left(5, str(self.is_leaf)),
             self.leaf_num)
             )
        if self.is_leaf:
            return
        else:
            for c in self.children:
                c.show(pad + 1)
class GeneralTree:
    def __init__(self, v):
        self.root = Node()
        self.root.value = v
        self.root.is_leaf = False
    
    def insert(self, value_l):
        r = self.root
        
        for value in value_l:
            for child in r.children:
                if value == child.value:
                    r = child
                    break
            else:
                r.insert(value)
                r = r.children[-1]
        r.is_leaf = True
        r.leaf_num += 1
    
    def show(self):
        self.root.show(1)
        
    

In [63]:
def IL_calc(org_l, mod_l, attr_list):
    # sequential numberをintに
    for org_r in org_l:
        org_r.append(int(org_r.pop()))
    for mod_r in mod_l:
        mod_r.append(int(mod_r.pop()))
    
    # sort by sequential number
    org_l.sort(key=lambda x: x[-1])
    mod_l.sort(key=lambda x: x[-1])
    
    # insert empty record to mod_l
    for i, mod_r in enumerate(mod_l):
        if i != mod_r[-1]:
            mod_l.insert(i, ['empty', i])
    
    # IL of addr
    addr_first, addr_last = addr_operation.addr_range_catcher(attr_list)
    
    ## get original addr list
    org_addr_l = [x[addr_first:addr_last+1] for x in org_l]
    
    ## get modified addr list
    mod_addr_l = list()
    for mod_r in mod_l:
        if mod_r[0] == 'empty':
            mod_addr_l.append(['関東', '*', '*', '*', '*'])
        else:
            mod_addr_l.append(mod_r[addr_first:addr_last+1])
        
    ## address tree from original addr list
    addr_tree = GeneralTree('関東')
    
    addr_attr = ['addr0', 'addr1', 'addr2', 'addr3', 'addr4']
    
    for org_addr in org_addr_l:
        org_addr = deepcopy(org_addr)
        
        general_l = list()
        prev = org_addr
        while(True):
            now = anonymizer.address_masking(addr_attr, deepcopy(prev))
            
            str_prev = ''.join(prev).strip('*')
            str_now = ''.join(now).strip('*')
            
            for chunk in str_now:
                str_prev = str_prev[1:]
            general_l.append(str_prev)
            
            if now.index('*') == 1:
                general_l.append(now[0])
                break
            
            prev = now
        
        general_l = general_l[::-1]
    
        addr_tree.insert(general_l)
    
    # addr_tree.show()
    
    ## IL calculation
    '''
    for i, mod_addr in enumerate(mod_addr_l):
        org_addr = org_addr_l[i]
        numerator = loss('addr', addr_tree_, org_addr)
        denominator = numerator + loss('addr', addr_tree_, mod_addr, org_addr)
        IL = numerator / denominator
    '''
    
    return addr_tree

In [64]:
addr_tree = IL_calc(org_list, anonymized_list, attr_list)